In [1]:
# initial imports

import h5py
import numpy as np
import pickle
from PIL import Image
from npy_append_array import NpyAppendArray
import random

import torchvision
from torchvision import transforms
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch.optim as optim

'''
from scipy.stats import zscore
import matplotlib as mpl
from matplotlib import pyplot as plt
'''

'\nfrom scipy.stats import zscore\nimport matplotlib as mpl\nfrom matplotlib import pyplot as plt\n'

In [2]:
# Set type
type = 'rand'

In [3]:
# Initialize model

if type == 'rand':
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=None)
elif type == 'model':
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=ResNet50_Weights.DEFAULT)
elif type == 'barlow':
    model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
elif type == 'model34':
    model = torchvision.models.resnet34(pretrained=True)
elif type == 'model18':
    model = torchvision.models.resnet18(pretrained=True)

model.eval()

Using cache found in /Users/parsatalaie/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# def hook

def hook(module, input, output):
    conv_outputs[i] = output[0].flatten().unsqueeze(0)

handle = model.layer4[0].conv2.register_forward_hook(hook)
#handle = model.avgpool.register_forward_hook(hook)

layer = 'conv4'
sublayer = '02'
size = 512*7*7 #for 402

In [5]:
# Load the data
pathname = '/Users/parsatalaie/Downloads/ventral_neural_data.hdf5'
f = h5py.File(pathname, 'r')
dset_images = f['images'][:]
dset_labels = f['image_meta']['object_name'][:]
meta = f['image_meta']
dset_neural = f['time_averaged_trial_averaged'][:]

In [6]:
# fix image, use model
trials = 1000

model_outputs = torch.zeros(1, 1000)
conv_outputs = torch.zeros(trials, size)

for i in range(trials):
    img = dset_images[i]
    input_image = Image.fromarray(img).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    with torch.no_grad():
        output = model(input_batch)
    if layer == 'final':
        model_outputs = torch.cat((model_outputs, output[0].unsqueeze(0)), dim=0)

if layer == 'final':
    outputs_np = model_outputs.detach().cpu().numpy()
else:
    outputs_np = conv_outputs.detach().cpu().numpy()

outputs_np.shape

(1000, 25088)

In [7]:
# INITIAL CREATION

np.save(f'./cadieu_data/cadieu_{type}_{layer}{sublayer}outputs', outputs_np)

In [5]:
# append outputs_np

filename = '/Users/parsatalaie/Desktop/Marmoset Data Analysis/new_marm_model_outputs.npy'

with NpyAppendArray(filename) as npaa:
    npaa.append(outputs_np)
    
data = np.load(filename, mmap_mode="r")

data.shape

(3364, 1000)

In [3]:
# Initial model weight save

torch.save(model.state_dict(), 'rand_weights.pth')

In [53]:
# Get trial counts for each image in train list

img_count = len(set(natimg_path_list))

count_arr = np.zeros((natimg_path_list.count(max(natimg_path_list)), img_count))
count_arr[:] = np.nan

for i in range(img_count):
    path = f'/Users/parsatalaie/Downloads/rust_natimgs/Nat300_{i+1}.png'
    idx_list = [i for i, j in enumerate(natimg_path_list) if j == path]
    for x, idx in enumerate(idx_list):
        count_arr[x, i] = idx

In [55]:
# Save count_arr

np.save(f'./{year}_{month}_{day}/img_idx_{year}{month}{day}', count_arr)

In [6]:
# Convert to png

for i in range(5760):
    # Convert the matrix to an image
    image = Image.fromarray(dset_images[i])
    categ = meta['category'][i]
    name = meta['object_name'][i]
    rot_xy = meta['rotation_xy'][i]
    rot_yz = meta['rotation_yz'][i]
    size = meta['size'][i]
    trans_y = meta['translation_y'][i]
    trans_z = meta['translation_z'][i]
    var = meta['variation_level'][i]
    
    # Save the image
    image.save(f"/Users/parsatalaie/Desktop/HvM Images/{i}_{categ}_{name}_{rot_xy}_{rot_yz}_{size}_{trans_y}_{trans_z}_{var}.png")

FileNotFoundError: [Errno 2] No such file or directory: "/Users/parsatalaie/Desktop/HvM Images/0_b'Fruits'_b'Apple_Fruit_obj'_-0.0_-0.0_0.81_0.0_0.0_b'V0'.png"

In [ ]:
# OLD MODEL LOOP
model_outputs = torch.zeros(1, 1000)

for i in range(3000, 3364):
    img = Image.open(natimg_path_list[i]).convert('RGB')
    img_np = np.transpose(np.asarray(img))
    img_tens = torch.from_numpy(img_np.astype('float32')).unsqueeze(0)
    img_transformed = preprocess(img_tens)
    output = model(img_transformed)
    model_outputs = torch.cat((model_outputs, output), dim=0)

outputs_np = model_outputs.detach().cpu().numpy()
outputs_np = np.delete(outputs_np, 0, 0)

outputs_np.shape